In [33]:
from functools import cached_property
from contextlib import AsyncExitStack, asynccontextmanager

class A:
    stack = AsyncExitStack()

    async def init_logic(self):
        async with self.stack:
            print("A enter")
            yield
            print("A exit")

    @cached_property
    def init_ctx(self):
        return asynccontextmanager(self.init_logic)()

class B(A):
    @asynccontextmanager
    async def blah(self):
        print("Blah B")
        yield
        print("Blub B")

    async def init_logic(self):
        generator = super().init_logic()
        async with self.stack:
            await self.stack.enter_async_context(self.blah())

        async for v in generator:
            yield v

class C(A):
    @asynccontextmanager
    async def blub(self):
        print("Blah C")
        yield
        print("Blub C")

    async def init_logic(self):
        generator = super().init_logic()
        async with self.stack:
            await self.stack.enter_async_context(self.blub())

        async for v in generator:
            yield v

class D(B, C):
    pass

In [34]:
import asyncio

async def run():
    x = D()
    async with x.init_ctx:
        print("Run")

await run()


Blah B
Blub B
Blah C
Blub C
A enter
Run
A exit


In [59]:
class InitContextManager:
    init_lock = asyncio.Lock()

    async def init_logic(self):
        if self.init_lock.locked():
            print("locked")
            yield self
            return
        else:
            async with self.init_lock:
                print("aquired lock")
                yield self

    @asynccontextmanager
    async def initialize(self):
        async for v in self.init_logic():
            yield v


In [62]:
class Kram(InitContextManager):
    init_lock = asyncio.Lock()
    @asynccontextmanager
    async def some_shit(self):
        if self.init_lock.locked():
            yield
        else:
            async with self.init_lock:
                print("Enter Zeug")
                yield
                print("Exit Zeug")

    def init_logic(self):
        async with self.some_shit():
            async for v in super().init_logic():
                yield v

class Zeug(InitContextManager):

    init_lock = asyncio.Lock()
    @asynccontextmanager
    async def other_shit(self):
        if self.init_lock.locked():
            yield
        else:
            async with self.init_lock:
                print("Enter Zeug")
                yield
                print("Exit Zeug")

    def init_logic(self):
        async with self.other_shit():
            async for v in super().init_logic():
                yield v

class Ding(Kram, Zeug):
    pass

async def run():
    x = Ding()
    async with x.initialize() as b:
        async with b.initialize() as c:
            print(c)

await run()

Enter Zeug
locked
locked
Exit Zeug
